In [26]:
##################################################################

# IMPORTANT:
# Run all cells up to 3 times, because it is 
# not guaranteed that all rows are intercepted the first time.

##################################################################

In [27]:
import csv
import re
from collections import OrderedDict

In [28]:
# counts number of rows
with open('blz-aktuell-xls-data.csv', "r", encoding="utf-8") as file:
    readCSV = csv.reader(file)
    lines = len(list(readCSV))
    
file.close()

In [29]:
with open('blz-aktuell-xls-data.csv', "r", encoding="utf-8") as file:
    readCSV = csv.reader(file)
    banksMarkedAlt = []
    
    # save banks with "-alt-" in bank name
    for row in readCSV:
        bankName = row[1]
        if "-alt-" in bankName or "-alt" in bankName:
            banksMarkedAlt += [row]
    
    file.close()
    
with open('blz-aktuell-xls-data.csv', "r", encoding="utf-8") as file:
    readCSV = csv.reader(file)
    
    removedRedundancy = open('removedAlt.csv', 'w')
    writer = csv.writer(removedRedundancy)
    
    counter = 0
    
    # don't write to file if "-alt-" 
    # or same bankcode from "-alt-" in bank name
    for line in readCSV:
        counter = 0
        
        if banksMarkedAlt:
            for bank in banksMarkedAlt:
                if not line[5] in bank[5]:
                    counter = counter + 1
                if counter == len(banksMarkedAlt):
                    writer.writerow(line)                    
        
    file.close()

In [30]:
def saveSameBankName(equalBanks, rowBefore, row):
    if not equalBanks:
            equalBanks += [rowBefore]
    equalBanks += [row]
    return equalBanks

def saveSameBICs(equalBanks, presentBIC):
    for line in equalBanks: 
        if line[2] != "":             
            presentBIC += [line]
    
    return presentBIC

def leaveOutRedundantLine(equalBanks, presentBIC):
    differentBankCodeBuffer = []
    
    for line in equalBanks:
        countPrintToFile = 0
        differentBankCode = 0
        counterDifferentBankCode = 1
        counterSameBankCode = 0

        # there is at least one BIC
        if presentBIC: 
            for bicLine in presentBIC:
                if countPrintToFile == 1:
                    continue
                elif line[5] == bicLine[5] and line[2] != "":   # bankcode same, BIC not empty
                    countPrintToFile = 1                        # also: different bankcode, BIC not empty
                    writer.writerow(line)
                elif line[5] == bicLine[5] and line[2] == "":   # bankcode same, BIC empty
                    countPrintToFile = 1
                elif line[5] != bicLine[5] and line[2] == "":   # bankcode different, BIC empty
                    differentBankCode = differentBankCode + 1

                if differentBankCode == len(presentBIC):
                    differentBankCodeBuffer += [line]

        # There is no BIC
        else:
            
            if len(equalBanks) == 1:
                writer.writerow(line)
            else:
                for lineAfter in equalBanks:
                    if line[5] != lineAfter[5]:
                        differentBankCodeBuffer += [line]
                    else:
                        counterSameBankCode = counterSameBankCode + 1
                    
                    if counterSameBankCode == len(equalBanks):
                        differentBankCodeBuffer += [line]
        

    # doesn't print duplicates from "bankcode different"
    if differentBankCodeBuffer != "":
        differentBankCodeBuffer = OrderedDict((tuple(x), x) for x in differentBankCodeBuffer).values()
        
        for items in differentBankCodeBuffer:
            writer.writerow(items)

        differentBankCodeBuffer = []

In [32]:
with open('removedAlt.csv', "r", encoding="utf-8") as file:
    readCSV = csv.reader(file)
    
    # output will be saved in new file
    noRedundancy = open('removedRedundancy.csv', 'w')
    writer = csv.writer(noRedundancy)
    bankName = ""; bic = ""; bankCode = ""; rowBefore = "" 
    equalBanks = []
    counter = 0
    presentBIC = []
  
    for row in readCSV:
        counter = counter + 1
        placeOfSecondComma = 0
        counterForLetter = 1
        newRowBuffer = ""
        counterForComma = 0
            
        if not equalBanks:
            if rowBefore:
                equalBanks = [rowBefore]
        
        # same bank name
        if bankName == row[1]:
            equalBanks = saveSameBankName(equalBanks, rowBefore, row)
            
        elif bankName in row[1] and ("(Gf P2)" in row[1] or "(Gf P3)" in row[1]):
            equalBanks = saveSameBankName(equalBanks, rowBefore, row)
        
        else:
            if "(Gf P2)" in bankName:
                bankName = re.sub("Gf P2", "", bankName)
                bankName = re.sub("[()]", "", bankName)
                
            elif "(Gf P3)" in bankName:
                bankName = re.sub("Gf P3", "", bankName)
                bankName = re.sub("[()]", "", bankName)
            
            # looks in array of same banks for redundancy
            if str(bankName) in str(equalBanks):
                presentBIC = saveSameBICs(equalBanks, presentBIC)
                leaveOutRedundantLine(equalBanks, presentBIC)

            # different bank names
            else: 
                writer.writerow(rowBefore)

            equalBanks = []    
            presentBIC = []
               
        bankName = row[1]
        bic = row[2]
        bankCode = row[5]
        rowBefore = row
        
        
         # handle last row
        if counter == lines:
            if re.search(bankName, str(equalBanks)):    
                presentBIC = saveSameBICs(equalBanks, presentBIC)
                leaveOutRedundantLine(equalBanks, presentBIC)

            # different bank names
            else: 
                writer.writerow(rowBefore)
                print("rowBefore")

            equalBanks = [] 
            presentBIC = []

            
    noRedundancy.close()
    file.close()